In [ ]:
import sys
import time
sys.path.append("../main") # go to main dir

from TrazeMqttAdapter import TrazeMqttAdapter

In [ ]:
import random

class RandomBot:
    def __init__(self):
        self._mqttAdapter = TrazeMqttAdapter("HansWurst")
        self._gameName, playerCount = next(iter(self._mqttAdapter.games().items()))
        self._lastDir = 'N'
        
    def __onUpdate__(self, player):
        directions = {'N' : (0, 1), 'S' : (0, -1), 'E' : (1, 0), 'W' : (-1, 0)}
        validDirs = list()
        for dir, delta  in directions.items():
            dX, dY = delta
            x, y = player._x + dX, player._y + dY
            value = player._tiles[y][x]
            print("  check for", player._x, player._y, 'at', x, y, 'has value', value)
            if (value == 0 and x >= 0 and x <= 60 and y >= 2 and y <= 61):
                validDirs.append(dir)

        if validDirs: 
            if self._lastDir not in validDirs:
                self._lastDir = random.choice(validDirs)

            dX, dY = directions[self._lastDir]
            x, y = player._x + dX, player._y + dY
            print("steer from", player._x, player._y, 'to', x, y, '(' + self._lastDir + ')')
            player.steer(self._lastDir)
        else:
            print("bail")
            player.bail()
            
    def play(self):
        player = self._mqttAdapter.join(self._gameName)
        player.subscribe(self.__onUpdate__, wait = True)

        # wait for death
        while(player.isAlive()):
           time.sleep(0.5)
                
    def die(self):
        self._mqttAdapter.destroy()        

In [ ]:
bot = RandomBot()
bot.play()

In [ ]:
bot.die()